## Load Data and Tokenizer

In [1]:
from transformers import BertTokenizer
from datasets import load_dataset

ds = load_dataset('billingsmoore/tagged-tibetan-to-english-translation-dataset')

tokenizer = BertTokenizer.from_pretrained('bert-large-cased')

## Preprocess Data

### Use just first two tags

In [2]:
def just_two_tags(examples):
    tags = [tag[:2] for tag in examples['Tags']]
    examples['Tags'] = tags
    return examples

ds = ds.map(just_two_tags, batched=True)

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()
labels = mlb.fit(ds['train']['Tags'])  # Fit all unique Tags

# Save label mappings
import json
with open("en_lh_label_mapping.json", "w") as f:
    json.dump(mlb.classes_.tolist(), f)


In [4]:
def preprocess(examples):
    tokens = tokenizer(examples["English"], padding="max_length", truncation=True, max_length=128)
    tokens["labels"] =  mlb.transform(examples['Tags']).astype(float).tolist() # Convert labels to multi-hot
    return tokens

encoded_dataset = ds.map(preprocess, batched=True)


Map:   0%|          | 0/107525 [00:00<?, ? examples/s]

In [5]:
encoded_dataset = encoded_dataset.remove_columns(['Tibetan', 'Phonetic', 'English', 'Tags'])

In [6]:
encoded_dataset = encoded_dataset['train'].train_test_split(.15)

In [7]:
"""enc = tokenizer(ds['train'][0]['Tibetan'])
tokenizer.decode(enc.input_ids)"""

"enc = tokenizer(ds['train'][0]['Tibetan'])\ntokenizer.decode(enc.input_ids)"

## Train Model

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load tokenizer and model
model = BertForSequenceClassification.from_pretrained("bert-large-cased", num_labels=len(mlb.classes_))

# Resize embeddings to match the new tokenizer
model.resize_token_embeddings(len(tokenizer))

# Move model to GPU
model = model.to('cuda:0')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    predictions, references = eval_pred
    
    # Apply a threshold to convert logits to binary predictions
    predictions = (predictions > 0.5).astype(int)
    
    # Compute metrics
    accuracy = accuracy_score(references, predictions)
    f1 = f1_score(references, predictions, average="micro")
    precision = precision_score(references, predictions, average="micro")
    recall = recall_score(references, predictions, average="micro")

    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision_1": precision,
        "recall": recall,
    }


In [10]:
from transformers import TrainingArguments, Trainer

from transformers import EarlyStoppingCallback

# Define training arguments
training_args = TrainingArguments(
    output_dir="en-lh-lg-bert-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,  # Set a maximum number of epochs
    weight_decay=0.01,
    eval_strategy="epoch",  # Evaluate at the end of every epoch
    save_strategy="epoch",  # Save the model at the end of every epoch
    load_best_model_at_end=True,  # Load the best model after training
    metric_for_best_model="accuracy",  # Metric to monitor
    greater_is_better=True,  # Higher accuracy is better
    logging_dir="./logs"
)

# Add the EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop training if the metric does not improve for 3 evaluation steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]  # Add the early stopping callback
)

# Start training
trainer.train()

/tmp/ipykernel_17507/1710218490.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: billingsmoore. Use `wandb login --relogin` to force relogin


  0%|          | 0/571300 [00:00<?, ?it/s]

{'loss': 0.1355, 'grad_norm': 0.06919872015714645, 'learning_rate': 1.9982496061613864e-05, 'epoch': 0.09}
{'loss': 0.0542, 'grad_norm': 0.06292929500341415, 'learning_rate': 1.996499212322773e-05, 'epoch': 0.18}
{'loss': 0.0524, 'grad_norm': 0.07311862707138062, 'learning_rate': 1.994748818484159e-05, 'epoch': 0.26}
{'loss': 0.0518, 'grad_norm': 0.06628433614969254, 'learning_rate': 1.9929984246455454e-05, 'epoch': 0.35}
{'loss': 0.0506, 'grad_norm': 0.0655488520860672, 'learning_rate': 1.9912480308069317e-05, 'epoch': 0.44}
{'loss': 0.0495, 'grad_norm': 0.0706687867641449, 'learning_rate': 1.9894976369683182e-05, 'epoch': 0.53}
{'loss': 0.0482, 'grad_norm': 0.07596486061811447, 'learning_rate': 1.987747243129704e-05, 'epoch': 0.61}
{'loss': 0.0474, 'grad_norm': 0.10036158561706543, 'learning_rate': 1.9859968492910907e-05, 'epoch': 0.7}
{'loss': 0.0472, 'grad_norm': 0.11987017095088959, 'learning_rate': 1.984246455452477e-05, 'epoch': 0.79}
{'loss': 0.045, 'grad_norm': 0.1358622312545

  0%|          | 0/1009 [00:00<?, ?it/s]

{'eval_loss': 0.04201948642730713, 'eval_accuracy': 0.0883501767003534, 'eval_f1': 0.18714687299952684, 'eval_precision_1': 0.915826750204304, 'eval_recall': 0.10422220844441689, 'eval_runtime': 112.7539, 'eval_samples_per_second': 143.046, 'eval_steps_per_second': 8.949, 'epoch': 1.0}
{'loss': 0.0425, 'grad_norm': 0.20416289567947388, 'learning_rate': 1.978995273936636e-05, 'epoch': 1.05}
{'loss': 0.0411, 'grad_norm': 0.13094212114810944, 'learning_rate': 1.9772448800980222e-05, 'epoch': 1.14}
{'loss': 0.0399, 'grad_norm': 0.16791507601737976, 'learning_rate': 1.9754944862594084e-05, 'epoch': 1.23}
{'loss': 0.0384, 'grad_norm': 0.16840516030788422, 'learning_rate': 1.973744092420795e-05, 'epoch': 1.31}
{'loss': 0.038, 'grad_norm': 0.1695626676082611, 'learning_rate': 1.9719936985821812e-05, 'epoch': 1.4}
{'loss': 0.0372, 'grad_norm': 0.16167773306369781, 'learning_rate': 1.9702433047435675e-05, 'epoch': 1.49}
{'loss': 0.0365, 'grad_norm': 0.20023199915885925, 'learning_rate': 1.968492

...

  0%|          | 0/1009 [00:00<?, ?it/s]

{'eval_loss': 0.012843534350395203, 'eval_accuracy': 0.7952135904271809, 'eval_f1': 0.871871678423036, 'eval_precision_1': 0.9072596862850248, 'eval_recall': 0.8391406782813565, 'eval_runtime': 113.3802, 'eval_samples_per_second': 142.256, 'eval_steps_per_second': 8.899, 'epoch': 25.0}
{'loss': 0.0027, 'grad_norm': 0.09436175227165222, 'learning_rate': 1.4993873621564854e-05, 'epoch': 25.03}
{'loss': 0.0024, 'grad_norm': 0.028055889531970024, 'learning_rate': 1.4976369683178718e-05, 'epoch': 25.12}
{'loss': 0.0026, 'grad_norm': 0.012291422113776207, 'learning_rate': 1.4958865744792578e-05, 'epoch': 25.21}
{'loss': 0.0029, 'grad_norm': 0.19876140356063843, 'learning_rate': 1.4941361806406442e-05, 'epoch': 25.29}
{'loss': 0.0028, 'grad_norm': 0.04795808345079422, 'learning_rate': 1.4923857868020306e-05, 'epoch': 25.38}
{'loss': 0.0028, 'grad_norm': 0.030622493475675583, 'learning_rate': 1.490635392963417e-05, 'epoch': 25.47}
{'loss': 0.0027, 'grad_norm': 0.13271339237689972, 'learning_ra

  0%|          | 0/1009 [00:00<?, ?it/s]

{'eval_loss': 0.012929117307066917, 'eval_accuracy': 0.7975695951391902, 'eval_f1': 0.8742945596439743, 'eval_precision_1': 0.9109879032258065, 'eval_recall': 0.8404426808853618, 'eval_runtime': 113.4414, 'eval_samples_per_second': 142.179, 'eval_steps_per_second': 8.894, 'epoch': 26.0}
{'loss': 0.0024, 'grad_norm': 0.04990154877305031, 'learning_rate': 1.478382636093121e-05, 'epoch': 26.08}
{'loss': 0.0026, 'grad_norm': 0.27397915720939636, 'learning_rate': 1.4766322422545074e-05, 'epoch': 26.17}
{'loss': 0.0026, 'grad_norm': 0.01916823908686638, 'learning_rate': 1.4748818484158938e-05, 'epoch': 26.26}
{'loss': 0.0027, 'grad_norm': 0.1451733410358429, 'learning_rate': 1.4731314545772799e-05, 'epoch': 26.34}
{'loss': 0.0026, 'grad_norm': 0.1396797001361847, 'learning_rate': 1.4713810607386663e-05, 'epoch': 26.43}


KeyboardInterrupt: 

## Test Outputs

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import json

# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("/home/j/Desktop/MLotsawa/Notebooks/Models/BertTag/bert-classifier/checkpoint-34275")
tokenizer = BertTokenizer.from_pretrained("./tibetan_tokenizer")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Load label mapping
with open("label_mapping.json", "r") as f:
    label_mapping = json.load(f)

# Example input text
input_text = "བླ་མ་དང་ལྷག་པའི་ལྷ་ལ་ཕྱག་འཚལ་ལོ།།"

# Tokenize input
encoded_input = tokenizer(
    input_text, 
    padding="max_length", 
    truncation=True, 
    max_length=128, 
    return_tensors="pt"
)
encoded_input = {key: val.to(device) for key, val in encoded_input.items()}

# Get predictions
with torch.no_grad():
    outputs = model(**encoded_input)
    logits = outputs.logits
    probabilities = torch.sigmoid(logits).cpu().numpy()
    predictions = (probabilities > 0.5).astype(int)

# Decode predictions
predicted_tags = [label_mapping[i] for i, val in enumerate(predictions[0]) if val == 1]

print("Predicted Tags:", predicted_tags)
